# 파이썬에서 한글 처리(1) 
---
- 한글 코드 변환
- 한글 호환 자모(Hangul Compatibility Jamo)
- unicode에서 한글 음절
- 한글 음절을 utf-8 코드로 encoding하는 방식
---
All rights reserved, 2021-2023 By **Youn-Sik Hong**. 수업 목적으로만 활용 가능.

- 파이썬에서는 utf-8 코드 체계를 사용    
    - utf-8 : unicode를 완벽하게 지원하는 코드 체계.
    - Unicode 문자코드 차트 : http://www.unicode.org/charts/
        - 한글은 East Asian Scripts의 Hangul Jamo에서 찾으면 됨.
- 파일을 읽어오거나 처리 결과를 파일로 저장할 때 문자열의 인코딩 방식에 맞는 변환이 필요.
    - decode : latin-2, utf-8 등으로 인코딩한 파일
        - 파이썬 내부로 가져올 때 unicode로 변환하는 과정.
    - encode : 파이썬 내부에서 처리한 unicode 문자열
        - utf-8, cp949 등으로 인코딩해서 파일로 저장하는 과정.

## 1. 한글 코드 변환 

In [ ]:
import sys

sys.getdefaultencoding()
#sys.getfilesystemencoding()

In [ ]:
print("홍".encode("euc-kr"))
#print("홍".encode("cp949"))
print("홍".encode("utf-8"))
print("홍".encode("utf-16"))

cp949는 완성형 코드를, utf-8(unicode)은 조합형 코드를 사용. 
- 아래에서 b는 binary, x는 hexadecimal(16진수)을 의미.

In [ ]:
#완성형 코드
print("홍".encode("cp949"), "can be decoded into", b"\xc8\xab".decode("cp949")) 

조합형 코드: 초성코드+중성코드+종성코드의 조합으로 한글 음절(syllable)을 구성.

In [ ]:
print("홍".encode("utf-8"), "can be decoded into", b'\xed\x99\x8d'.decode("utf-8")) 

**김 영랑** 시인의 시 '오매, 단풍들것네'
- *오매 단풍들것네 장광에 골불은 감닙 날러오아 누이는 놀란 듯이 치어다보며 오매 단풍들것네.*

In [ ]:
hangul_sent = '오매 단풍들것네' 
print(hangul_sent)

시스템 내부 또는 파일에 저장될 때 어떻게 표현될까? - \\uxxxx (x는 16진수)
- encode 메소드에서 **unicode_escape** 옵션을 설정하면 음절(syllable) 단위로 저장. 
- '오','매',' ','단','풍','들','것','네'와 같이 8개 음절.

In [ ]:
print(hangul_sent.encode('unicode_escape'))

- encode 메소드에서 **utf-8** 옵션을 설정하면, 
    - 한글 음절을 구성하는 **자모(JAMO**, 자음모음) 단위로 저장.
---    
- '오'는 'ㅇ'+'ㅗ'+'받침 없음'과 같이 3개 코드로 변환되어 저장.
    - 모든 한글 음절은 각각 3개의 자모 코드로 변환. 
    - 7개 한글 음절이기 때문에 21개 코드로 바뀜.
    - \\x는 16진수를 표시하는 prefix이기 때문에 다음 2자리 16진수가 자모 코드.
---    
- space 문자는 화면에 보이는 그대로 저장됨. 
    - space 문자는 ASCII(7bits)의 128개 코드에 포함되기 때문.

In [ ]:
print(hangul_sent.encode('utf-8')) 
#print(hangul_sent.encode()) #default는 utf-8

In [ ]:
import unicodedata

- 아래 예에서 by, Young, Rang, Kim과 space 및 dash (-)문자는 
    - ASCII 코드에 포함되기 때문에 출력되지 않음.

In [ ]:
hangul_sent2 = '오매 단풍들것네 by Young-Rang Kim'
for c in hangul_sent2: 
    if ord(c) > 127:  #ASCII 코드가 아닌 경우 unicode로 변환
        print('{} U+{:04x} {}'.format(c.encode('utf8'), ord(c), unicodedata.name(c)))

- 초성코드+중성코드+종성코드로 만들어진 한글 음절과 
- unicode로 정의한 한글 음절의 디코딩 결과를 확인해 보자.

In [ ]:
print(b'\xec\x98\xa4'.decode('utf-8'), '\uc624')
#print(b'\xeb\xa7\xa4'.decode('utf-8'), '\ub9e4')

## 2. 한글 호환 자모(Hangul Compatibility Jamo)
- 자모(자음, 모음)만을 위한 코드이며, 자모가 정상 크기로 출력됨.
    - 자음 = 단자음 + 쌍자음 + 겹자음(받침).
    - 모음 = 단모음 + 겹모음

In [ ]:
def show_hangulcode(s, e, f=False):
    if f:
        print(s, e, e-s+1, hex(s), hex(e))
    print() #한글이 차지하는 공간이 넓어서 여유 공간(padding)을 확보
    for i in range(s, e):
        print(chr(i), end=' ')    

In [ ]:
start = int('3131', 16) #16진수를 10진수로 변환
end = int('314e', 16)
show_hangulcode(start, end)

In [ ]:
start = int('314f', 16) 
end = int('3163', 16)
show_hangulcode(start, end)

## 3. unicode에서 한글 음절
- unicode에서 한글 음절은 모두 19(초성)x21(중성)x28(종성) = 11,172자이며, 
    - 첫 음절은 '가', 마지막 음절은 '힣'.
- 첫 음절 '가'의 코드는 44032(0xAC00), 마지막 음절 '힣'의 코드는 55203(0xD7A3).
- 초성 시작 위치: 0x1100(4352)
- 중성 시작 위치: 0x1161(4449)
- 종성 시작 위치: 0x11A7(4519)

In [ ]:
a,b,c = int('1100', 16), int('1161', 16), int('11A7', 16)
a, b, c

unicode에서 한글 초성: 19자 = 홑자음 14자 + 쌍자음 5자

In [ ]:
start = int('1100', 16) 
end = start+19
print()
show_hangulcode(start, end)

unicode에서 한글 중성: 21자 = 홑모음 10자 + 겹모음 11자

In [ ]:
start = int('1161', 16) 
end = start+21
print()
show_hangulcode(start, end)

unicode에서 한글 종성: 28자 = 받침없음(1자) + 자음 14자 + 쌍자음 2자 + 겹자음 11자

In [ ]:
start = int('11a7', 16) 
end = start+28
print()
show_hangulcode(start+1, end) #공백 문자를 출력하지 않음.

**ord()**: 한글 음절에 할당된 unicode를 정수로 출력

In [ ]:
print(ord('가'), ord('힣')) 

**hex()**: 10진수를 16진수로 변환
**chr()**: 정수 코드를 문자로 변환

In [ ]:
print(chr(44032), hex(44032)) 
print(chr(55203), hex(55203)) 

In [ ]:
han_syllable_1 = '\uac00' #unicode를 문자열(str)로 변환
han_syllable_11172 = '\ud7a3'
print(type(han_syllable_1))
print(han_syllable_1, han_syllable_11172)

In [ ]:
print('\uac00'.encode('utf8'), '가'.encode('utf8'))
print('\ud7a3'.encode('utf8'), '힣'.encode('utf8'))

## 4. 한글 음절을 utf-8 코드로 encoding하는 방식
- 한글 음절(2 bytes) $\rightarrow$  utf-8(3 bytes)
- utf-8의 3 bytes 중 16 bits(= 2 bytes)가 비어 있음
    - 1번째 바이트: 1110 xxxx (4bits 여유 공간)
    - 2번째 바이트: 10xx xxxx (6bits 여유 공간)
    - 3번째 바이트: 10xx xxxx (6bits 여유 공간) 
- 한글 음절(2 bytes)를 순차적으로 16 bits 공간에 배치  
    - 1번째 바이트: xxxx xxxx = h1 h2
    - 2번째 바이트: xx xxxxxx = h3 h4
- utf-8 코드로 encoding
    - 1110 h1(4bits) $\rightarrow$  1번째 바이트 
    - 10 h2(4bits) h3(2bits) $\rightarrow$  2번째 바이트 
    - 10 h3(6bits) $\rightarrow$  3번째 바이트 

In [ ]:
first_utf8 = 0b11100000
second_utf8 = 0b10000000
third_utf8 = 0b10000000

high_가 = 0xAC; low_가 = 0x00
high_힣 = 0xD7; low_힣 = 0xA3

In [ ]:
h1 = high_가 >> 4
#h1 = high_힣 >> 4
print("h1=", bin(h1)[2:].zfill(4))

h2 = high_가 << 2
#h2 = high_힣 << 2
h2 = h2 & 0b0000111100
print("h2=", bin(h2)[2:].zfill(4))

In [ ]:
h3 = low_가 >> 6
#h3 = low_힣 >> 6
print("h3=", bin(h3)[2:].zfill(2))

h4 = low_가 & 0b00111111
#h4 = low_힣 & 0b00111111
print("h4=", bin(h2)[2:].zfill(6))

In [ ]:
utf1 = bin(first_utf8 | h1)
utf2 = bin(second_utf8 | h2 | h3)
utf3 = bin(third_utf8 | h4)
#print(utf1, utf2, utf3)
print(hex(int(utf1, 2)), hex(int(utf2, 2)), hex(int(utf3, 2)))

In [ ]:
'가'.encode("utf-8")
#'힣'.encode("utf-8")